In [7]:
## https://colab.research.google.com/drive/1iAE2OUx7nFZzyrasJMYGf8fxd_fO80t7

In [1]:
import string

import gensim
import numpy as np
import pandas as pd
import tensorflow as tf
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Embedding, SimpleRNN, LSTM, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
MAX_LEN = 300
NUM_WORDS = 40000
NUM_LABELS = 2

In [3]:
def filter_by_ascii_rate(text, threshold=0.9):
    ascii_letters = set(string.printable)
    rate = sum(c in ascii_letters for c in text) / len(text)
    return rate <= threshold


def load_dataset(filename, n=5000):
    df = pd.read_csv(filename, sep='\t')

    # Converts multi-class to binary-class.
    mapping = {1: 0, 2: 0, 4: 1, 5: 1}
    df = df[df.star_rating != 3]
    df.star_rating = df.star_rating.map(mapping)

    # extracts Japanese texts.
    is_jp = df.review_body.apply(filter_by_ascii_rate)
    df = df[is_jp]

    # sampling.
    df = df.sample(frac=1, random_state=7)  # shuffle
    grouped = df.groupby('star_rating')
    df = grouped.head(n=n)
    return df.review_body.values, df.star_rating.values


url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz'
x, y = load_dataset(url)

In [4]:
def load_fasttext(filepath, binary=False):
    """Loads fastText vectors.
    Args:
        filepath (str)
    Return:
        model: KeyedVectors
    """
    model = gensim.models.KeyedVectors.load_word2vec_format(filepath, binary=binary)
    return model
wv = load_fasttext('data/cc.ja.300.vec.gz')


In [5]:
t = Tokenizer(wakati=True)


def build_vocabulary(texts, num_words=None):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=num_words, oov_token='<UNK>'
    )
    tokenizer.fit_on_texts(texts)
    return tokenizer


def clean_html(html, strip=False):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(strip=strip)
    return text


def tokenize(text):
    return t.tokenize(text)


def preprocess_dataset(texts):
    # データセットからHTML等を除去
    texts = [clean_html(text) for text in texts]
    # janomeでトークン化した
    texts = [' '.join(tokenize(text)) for text in texts]
    return texts

# この関数何やっている？
def filter_embeddings(embeddings, vocab, num_words, dim=300):
  """Filter word vectors.

  Args:
      embeddings: a dictionary like object.
      vocab: word-index lookup table.
      num_words: the number of words.
      dim: dimension.

  Returns:
      numpy array: an array of word embeddings.
  """
  # num_words:文章における単語の数？それとも辞書に存在する単語の数？
  # 文章における単語っぽい..ではなくそれとも辞書に存在する単語の数みたいだ！
  # len(wv) === 40000だったので確定。
  _embeddings = np.zeros((num_words, dim))
  
  for word in vocab:
      if word in embeddings:
          word_id = vocab[word]
          if word_id >= num_words:
              continue
          _embeddings[word_id] = embeddings[word]

  return _embeddings

In [19]:
len(wv)

40000

In [6]:
wv.most_similar("嫌い",topn=10)

[('大嫌い', 0.8515045046806335),
 ('キライ', 0.7897728085517883),
 ('好き', 0.7893754243850708),
 ('苦手', 0.7790613770484924),
 ('嫌う', 0.7406898736953735),
 ('嫌', 0.7398313879966736),
 ('大好き', 0.6881823539733887),
 ('毛嫌い', 0.6711427569389343),
 ('嫌っ', 0.6663845777511597),
 ('嫌がる', 0.6618356704711914)]

In [44]:
X = preprocess_dataset(x)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
# ここ何やっている?
# 
vocab = build_vocabulary(x_train, NUM_WORDS)
# ここ何やっている? : idを割り当てている:https://qiita.com/tomiyou/items/da0b4cc85b89eb0b6d1d
# 学習済みモデルのIDを割り当てないで大丈夫なのか？
x_train = vocab.texts_to_sequences(x_train)
x_test = vocab.texts_to_sequences(x_test)

In [24]:
# ここ何やっている? ref:https://qiita.com/9ryuuuuu/items/2830fee559a41d00aa2b
x_train = pad_sequences(x_train, maxlen=MAX_LEN, truncating='post', padding='post')
x_test = pad_sequences(x_test, maxlen=MAX_LEN, truncating='post', padding='post')

wv = filter_embeddings(wv, vocab.word_index, NUM_WORDS)

/var/folders/8x/f7l0kj057qx9_nym27s1dgc40000gn/T/ipykernel_55145/1067934105.py:46: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if word in embeddings:


In [25]:
X

['子ども たち に 見せ た ところ 、 もう 、 ページ を めくる たび に 面白い ねぇ という 喜び の 声 が 聞か れ まし た よ 。 今 の 子ども たち は 草原 を 通り抜け たり 、 川 を 渡っ たり など 、 お話 に 出 て くる よう な 体験 が あまり あり ませ ん 。 です から 、 この ポップアップ は ほんとに イメージ が 膨らん で くる ので いい なぁ と 思い ます 。 ぜひ 、 お 薦め です 。',
 '１ 枚 目 は 、 完全 に 演奏 は レイジ 、 メロディー は クリス と 、 水 と 油 みたい な 感じ で 違和感 たっぷり でし た が 、 ２ 枚 目 に なっ て 、 かなり クリス 色 の 強い バンド に 生まれ変わっ た 感じ が し ます 。 要は サウンド ガーデン っぽく なっ た 。 ただ 、 出し てる 音 は レイジ そのもの な ので 、 クリス の ボーカル スタイル に は 少し オケ が 寂しく 感じ て しまう 。 ７ 曲 目 くらい 、 ギター に も ボーカル に も エフェクター を 効か せ て 派手 に し て やっと クリスコーネル 節 が 生き て くる 。 スーパーアンノウン を 超える ほど の アルバム を 期待 し てる 私 として は また 肩透かし を 食らい まし た （ そりゃ 当人 たち は そんな 音楽 を する つもり は 無い の でしょ う が ） 冷静 に 、 サウンド ガーデン 、 レイジアゲインストザマシーン それぞれ の 一番 好き な アルバム と 比較 し て 、 それぞれ より も かっこいい と は 言え ませ ん 。 ただし 、 １ 枚 目 より ２ 枚 目 が 凄く 良い の は 確か な ので 、 ３ 枚 目 に 期待 を 込め て 、 この 作品 を 噛み締める という の が 、 レイジファン 、 クリス ファン の 楽しみ 方 な の かも しれ ませ ん 。 ８ 曲 目 とか １ ０ 曲 目 に 、 新しい 可能 性 を 感じ た 私 は そう し まし た 。',
 '最近 手 に 入れ た ばかり な ので 詳しく 聴い て ない の です が 、 単なる リマ スター で は なく 、 リ

In [26]:
x_train.shape

(8000, 300)

In [27]:
x_train[:10]

array([[    1,     1,     1, ...,     1,   332,     1],
       [    1, 11569,     1, ...,     0,     0,     0],
       [    1, 23670,     1, ...,     0,     0,     0],
       ...,
       [    1,     1,   332, ...,     0,     0,     0],
       [    1,   332,     1, ...,     0,     0,     0],
       [    1,     1,   332, ...,     0,     0,     0]], dtype=int32)

In [39]:
class RNNModel:

    def __init__(self, input_dim, output_dim,
                 emb_dim=300, hid_dim=100,
                 embeddings=None, trainable=True):
        self.input = Input(shape=(None,), name='input')
        if embeddings is None:
            self.embedding = Embedding(input_dim=input_dim,
                                       output_dim=emb_dim,
                                       mask_zero=True,
                                       trainable=trainable,
                                       name='embedding')
        else:
            self.embedding = Embedding(input_dim=embeddings.shape[0],
                                       output_dim=embeddings.shape[1],
                                       mask_zero=True,
                                       trainable=trainable,
                                       embeddings_initializer=tf.keras.initializers.Constant(embeddings),
                                       # weights=[embeddings],
                                       name='embedding')
        self.rnn = SimpleRNN(hid_dim, name='rnn')
        self.fc = Dense(output_dim, activation='softmax')

    def build(self):
        x = self.input
        embedding = self.embedding(x)
        output = self.rnn(embedding)
        y = self.fc(output)
        return Model(inputs=x, outputs=y)

In [40]:
# こういう風に予測用のクラスを定義するのか！
class InferenceAPI:
    """A model API that generates output sequence.

    Attributes:
        model: Model.
        vocab: language's vocabulary.
    """

    def __init__(self, model, vocab, preprocess):
        self.model = model
        self.vocab = vocab
        self.preprocess = preprocess

    def predict_from_texts(self, texts):
        x = self.preprocess(texts)
        x = self.vocab.texts_to_sequences(x)
        return self.predict_from_sequences(x)

    def predict_from_sequences(self, sequences):
        sequences = pad_sequences(sequences, truncating='post')
        y = self.model.predict(sequences)
        return np.argmax(y, -1)

In [42]:
model_path = 'models/model_{}'
embeddings = [None]
models = [RNNModel]
batch_size = 128
epochs = 100
i = 0
for model, embedding in zip(models, embeddings):
    tf.keras.backend.clear_session()
    model = model(NUM_WORDS, NUM_LABELS, embeddings=embedding).build()
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['acc']
    )

    callbacks = [
        EarlyStopping(patience=3),
        ModelCheckpoint(model_path.format(i), save_best_only=True)
    ]

    model.fit(
        x=x_train, y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        callbacks=callbacks,
        shuffle=True
    )
    i += 1

Epoch 1/100
50/50 [==============================] - 19s 363ms/step - loss: 0.6852 - acc: 0.5453 - val_loss: 0.6917 - val_acc: 0.5550
INFO:tensorflow:Assets written to: models/model_0/assets
Epoch 2/100
50/50 [==============================] - 18s 369ms/step - loss: 0.6520 - acc: 0.6141 - val_loss: 0.6830 - val_acc: 0.5744
INFO:tensorflow:Assets written to: models/model_0/assets
Epoch 3/100
50/50 [==============================] - 18s 369ms/step - loss: 0.6255 - acc: 0.6436 - val_loss: 0.6859 - val_acc: 0.5794
Epoch 4/100
50/50 [==============================] - 19s 381ms/step - loss: 0.5802 - acc: 0.6964 - val_loss: 0.6794 - val_acc: 0.6081
INFO:tensorflow:Assets written to: models/model_0/assets
Epoch 5/100
50/50 [==============================] - 19s 384ms/step - loss: 0.5405 - acc: 0.7237 - val_loss: 0.6807 - val_acc: 0.6150
Epoch 6/100
50/50 [==============================] - 19s 386ms/step - loss: 0.5223 - acc: 0.7294 - val_loss: 0.7185 - val_acc: 0.5713
Epoch 7/100
50/50 [======

In [43]:
model = load_model(model_path.format(0))

In [ ]:
api = InferenceAPI(model,vocab,preprocess_dataset)
y_pred = api.predict_from_sequences(x_test)

In [ ]:
print(precision_score(y_test, y_pred,average="binary"))
print(recall_score(y_test, y_pred,average="binary"))
print(f1_score(y_test, y_pred,average="binary"))